In [1]:
import os
import warnings
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import entropy
from sklearn.metrics.pairwise import cosine_similarity

from tqdm.notebook import tqdm


import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, SubsetRandomSampler, random_split, Dataset

import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models, transforms, models

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device

print(f'Application running on {device}')

Application running on cuda:0


In [3]:
class CIFAR10(Dataset):
    def __init__(self, bool_train, new_dataset=None, transform=None):
        self.cifar10 = new_dataset if bool_train == None else datasets.CIFAR10('./root', train=bool_train, download=True, transform=transform)

    def __len__(self):
        return len(self.cifar10)

    def __getitem__(self, index):
        image, label = self.cifar10[index]

        return index, image, label

In [4]:
transform_cifar10 = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 16

trainset = CIFAR10(bool_train=True, transform=transform_cifar10)

testset = CIFAR10(bool_train=False, transform=transform_cifar10)
test_dl = DataLoader(testset, batch_size, shuffle=True, num_workers=2, pin_memory=True)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

100%|██████████| 170498071/170498071 [00:05<00:00, 33621985.74it/s]


Extracting ./root/cifar-10-python.tar.gz to ./root
Files already downloaded and verified


In [5]:
def get_initial_dataloaders(trainset, val_rateo, labeled_ratio):

  train_size = len(trainset) #50000

  val_size = int(train_size * val_rateo)
  train_size -= val_size

  train_data, val_data = random_split(trainset, [int(train_size), int(val_size)])

  # validation dataloader
  val_dl = DataLoader(val_data, batch_size=batch_size, shuffle=True, num_workers=2)
  train_dl = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=2)
  #-------------

  train_data_size = len(train_data)

  # Calculate the number of samples for each split
  labeled_size = int(labeled_ratio * train_data_size)
  unlabeled_size = train_data_size - labeled_size

  # Get the dataset split
  labeled_set, unlabeled_set = random_split(train_data, [labeled_size, unlabeled_size])

  # Obtain the splitted dataloader
  labeled_train_dl = DataLoader(labeled_set, batch_size=batch_size, shuffle=True, num_workers=2)
  unlabeled_test_dl = DataLoader(unlabeled_set, batch_size=batch_size, shuffle=True, num_workers=2)

  return (labeled_train_dl, unlabeled_test_dl), (labeled_set, unlabeled_set), train_dl, val_dl

In [6]:
def get_overall_top_k(topk_list, top_k):

    # Concatenate all the top_k.values tensors
    all_values = torch.cat([topk_result.values for topk_result in topk_list])

    # Find the overall top k values and their corresponding indices
    overall_topk_values, overall_topk_indices = torch.topk(all_values, k=top_k)

    overall_top_k = []

    # Print the results
    for i in range(len(overall_topk_values)):
        index_position = overall_topk_indices[i].item()

        # Find the list index and index value within the specific top_k result
        list_index = 0
        current_values_count = 0
        for topk_result in topk_list:
            if index_position < current_values_count + len(topk_result.values):
                index_value = index_position - current_values_count
                break

            current_values_count += len(topk_result.values)
            list_index += 1

        overall_top_k.append((list_index, index_value))

    return overall_top_k

In [7]:
class GTG_ActiveLearning():


    def __init__(self, n_classes, model, optimizer, train_dl, test_dl, splitted_train_dl, splitted_train_ds, loss_fn, val_dl, score_fn, scheduler, device, patience):
        self.n_classes = n_classes
        self.model = model.to(device)
        self.optimizer = optimizer
        self.train_dl = train_dl
        self.test_dl = test_dl
        self.lab_train_dl, self.unlab_train_dl = splitted_train_dl
        self.lab_train_ds, self.unlab_train_ds = splitted_train_ds
        self.loss_fn = loss_fn
        self.val_dl = val_dl
        self.score_fn = score_fn
        self.scheduler = scheduler
        self.device = device
        self.patience = patience
        self.best_check_filename = '/content/best_checkpoint.pth.tar'
        self.init_check_filename = '/content/init_checkpoint.pth.tar'
        self.__save_checkpoint(self.init_check_filename)

        self.labeled_embeddings = torch.empty(0, list(self.model.children())[-1].in_features).to(self.device)
        self.unlabeled_embeddings = torch.empty(0, list(self.model.children())[-1].in_features).to(self.device)

        self.embed_model = nn.Sequential(*list(self.model.children())[:-1]).to(self.device)



    def __reintialize_model(self):
        self.__load_checkpoint(self.init_check_filename, 'Initial')



    def __save_checkpoint(self, filename):

        print(f'=> Saving Checkpoint to {filename}')
        checkpoint = { 'state_dict': self.model.state_dict(), 'optimizer': self.optimizer.state_dict(), 'scheduler': self.scheduler.state_dict() }
        torch.save(checkpoint, filename)
        print(' DONE\n')



    def __load_checkpoint(self, filename, type_load):

        print(f'=> Loading {type_load} Checkpoint')
        checkpoint = torch.load(filename, map_location=self.device)
        self.model.load_state_dict(checkpoint['state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer'])
        self.scheduler.load_state_dict(checkpoint['scheduler'])
        print(' DONE\n')



    def evaluate(self, val_dl, epoch = 0, epochs = 0):
        val_loss, val_accuracy = .0, .0

        self.model.eval()

        pbar = tqdm(enumerate(val_dl), total = len(val_dl), leave=False)

        with torch.inference_mode(): # Allow inference mode
            for _, (_, images, label) in pbar:
                images, label = images.to(self.device), label.to(self.device)

                output = self.model(images)

                loss = self.loss_fn(output, label)

                accuracy = self.score_fn(output, label)

                val_loss += loss.item()
                val_accuracy += accuracy

                if epoch > 0: pbar.set_description(f'EVALUATION Epoch [{epoch} / {epochs}]')
                else: pbar.set_description(f'TESTING')
                pbar.set_postfix(loss = loss.item(), accuracy = accuracy)

            val_loss /= len(val_dl) # Calculate the final loss
            val_accuracy /= len(val_dl)
        return val_loss, val_accuracy



    def fit(self, epochs):
        self.model.train()

        best_val_loss, best_train_accuracy = float('inf'), float('-inf')
        actual_patience = 0

        for epoch in range(epochs):  # loop over the dataset multiple times

            train_loss = 0.0
            train_accuracy = 0.0

            pbar = tqdm(enumerate(self.lab_train_dl), total = len(self.lab_train_dl), leave=False)

            for _, (_, inputs, labels) in pbar:
                # zero the parameter gradients
                self.optimizer.zero_grad()

                # get the inputs; data is a list of [inputs, labels]
                inputs, labels = inputs.to(self.device), labels.to(self.device)

                outputs = self.model(inputs)
                loss = self.loss_fn(outputs, labels)

                loss.backward()
                self.optimizer.step()

                accuracy = self.score_fn(outputs, labels)

                # print statistics
                train_loss += loss.item()
                train_accuracy += accuracy

                # Update the progress bar
                pbar.set_description(f'TRAIN Epoch [{epoch + 1} / {epochs}]')
                pbar.set_postfix(loss = loss.item(), accuracy = accuracy)

            train_loss /= len(self.lab_train_dl)
            train_accuracy /= len(self.lab_train_dl)

            self.scheduler.step(train_loss)

            # Validation step
            val_loss, val_accuracy = self.evaluate(self.val_dl, epoch + 1, epochs)

            print('Epoch [{}], train_loss: {:.6f}, train_accuracy: {:.6f}, val_loss: {:.6f}, val_accuracy: {:.6f} \n'.format(
                      epoch + 1, train_loss, train_accuracy, val_loss, val_accuracy))

            if(val_loss < best_val_loss):
                best_val_loss = val_loss
                actual_patience = 0
                self.__save_checkpoint(self.best_check_filename)
            else:
                actual_patience += 1
                if actual_patience >= self.patience:
                    print(f'Early stopping, validation loss do not decreased for {self.patience} epochs')
                    pbar.close() # Closing the progress bar before exiting from the train loop
                    break

        self.__load_checkpoint(self.best_check_filename, 'Best')

        print('Finished Training\n')



    def get_embeddings(self, mode_type, dataloader):

        print(f'{mode_type} Embedding Computation')

        if mode_type == 'Sampled':
            self.unlabeled_embeddings = torch.empty(0, list(self.model.children())[-1].in_features).to(self.device)
        else:
            self.labeled_embeddings = torch.empty(0, list(self.model.children())[-1].in_features).to(self.device)

        self.embed_model.eval()

        pbar = tqdm(enumerate(dataloader), total = len(dataloader), leave=False)

        # again no gradients needed
        with torch.inference_mode():
            for _, (_, inputs, labels) in pbar:
                pbar.set_description(f'Getting {mode_type} Embeddings')
                embed = self.embed_model(inputs.to(self.device))

                if mode_type == 'Labeled':
                    self.labeled_embeddings = torch.cat((self.labeled_embeddings, embed.squeeze()), dim=0)
                else:
                    self.unlabeled_embeddings = torch.cat((self.unlabeled_embeddings, embed.squeeze()), dim=0)

        print(' => DONE\n')



    def get_unlabeled_samples(self, n_split):
        self.unlab_samp_list = []

        new_unlab_train_ds = self.unlab_train_ds
        unlabeled_size = len(new_unlab_train_ds)
        sampled_unlab_size = int(unlabeled_size // n_split)

        while (unlabeled_size > 0):

            unlabeled_size -= sampled_unlab_size

            # here I have random sampled from the unalbeled observation, uo
            sampled_unlab_ds, new_unlab_train_ds = random_split(new_unlab_train_ds, [int(sampled_unlab_size), int(unlabeled_size)])

            self.unlab_samp_list.append((sampled_unlab_ds,
                                         DataLoader(sampled_unlab_ds, batch_size=batch_size, shuffle=False, num_workers=2)))




    def get_A(self, sigma = 1):
        print('Obtaining Affinity Matrix')
        print(self.labeled_embeddings.shape, self.unlabeled_embeddings.shape)
        embeddings_cat = torch.cat((self.labeled_embeddings, self.unlabeled_embeddings), dim=0).to(self.device)

        # Computing Cosaine Similarity -> PROBELMATIC
        #self.A = F.cosine_similarity(embeddings_cat[None,:,:], embeddings_cat[:,None,:], dim=-1)
        #normalized_embedding = F.normalize(self.embeddings, p=2, dim=-1).to(self.device)
        #self.A = torch.matmul(normalized_embedding, normalized_embedding.transpose(-1, -2)).to(self.device)

        # Calculate Gaussian kernel
        self.A = torch.exp(-(torch.cdist(embeddings_cat, embeddings_cat)).pow(2) / (2.0 * sigma**2))

        # Calculate the Euclidean Distance
        #self.A = torch.cdist(embeddings_cat, embeddings_cat).to(self.device)
        print(self.A.shape)
        print(f' => DONE with memory usage {self.A.element_size() * self.A.nelement()} bytes\n')



    def get_X(self):
        print('Obtaining Initial X Matrix')

        self.X = torch.empty(0, self.n_classes).to(self.device)

        # X for the labeled observations
        for (_, _, label) in self.lab_train_ds:
            arr_one_zeros = torch.zeros(1, self.n_classes).to(self.device)
            arr_one_zeros[0][label] = 1
            self.X = torch.cat((self.X, arr_one_zeros), dim=0)

        self.X = torch.cat((self.X, torch.full((len(self.unlabeled_embeddings), self.n_classes), 1/self.n_classes).to(self.device)), dim=0)
        print(self.X.shape)
        print(f' => DONE with memory usage {self.X.element_size() * self.X.nelement()} bytes\n')



    def gtg(self, tol, max_iter):
        err = float('Inf')
        i = 0

        print('Runnning GTG Algorithm')

        while err > tol and i < max_iter:
            X_old = self.X.clone()
            self.X = self.X * torch.mm(self.A, self.X)
            self.X /= self.X.sum(axis=1, keepdim=True)

            err = torch.norm(self.X - X_old)
            i += 1

        if i == max_iter:
            warnings.warn('Max number of iterations reached.')

        print(f' => DONE with {i} iterations\n')

        return i



    def get_topK_obs(self, top_k):
        print('Obtaining the top_k most interesting observations')

        self.topk_idx_val_obs = torch.topk(-torch.sum(self.X * torch.log2(self.X + 1e-20), dim=1), top_k)

        print(' => DONE\n')



    def get_new_dataloader(self, overall_topk, n_samples):

        print('Copying the labeled train dataset')
        new_lab_train_ds = np.array([
            np.array([
                self.lab_train_ds[i][1] if isinstance(self.lab_train_ds[i][1], np.ndarray) else self.lab_train_ds[i][1].numpy(),
                self.lab_train_ds[i][2]
            ], dtype=object) for i in tqdm(range(len(self.lab_train_ds)))], dtype=object)
        #new_lab_train_ds = torch.tensor([self.lab_train_ds[i][::1]] for i in tqdm(range(len(self.lab_train_ds))))
        print(' => DONE\n')

        print('Copying the unlabeled train dataset')
        new_unlab_train_ds = np.array([
            np.array([
                self.unlab_train_ds[i][1] if isinstance(self.unlab_train_ds[i][1], np.ndarray) else self.unlab_train_ds[i][1].numpy(),
                self.unlab_train_ds[i][2]
            ], dtype=object) for i in tqdm(range(len(self.unlab_train_ds)))], dtype=object)
        #new_unlab_train_ds = torch.tensor([self.unlab_train_ds[i][::1]] for i in tqdm(range(len(self.unlab_train_ds))))
        print(' => DONE\n')


        l = len(new_lab_train_ds)

        print(f'Expanding the labeled train dataset {len(new_lab_train_ds)}')
        for list_index, topk_index_value in overall_topk:
            new_lab_train_ds = np.vstack((new_lab_train_ds, np.expand_dims(
                np.array([
                    self.unlab_samp_list[list_index][0][topk_index_value][1] if isinstance(self.unlab_samp_list[list_index][0][topk_index_value][1], np.ndarray)
                        else self.unlab_samp_list[list_index][0][topk_index_value][1].numpy(),
                    self.unlab_samp_list[list_index][0][topk_index_value][2]
                ], dtype=object)
            , axis=0)))
            #new_lab_train_ds = torch.cat((new_lab_train_ds, torch.tensor([self.unlab_samp_list[list_index][0][topk_index_value][::1]])), dim = 0)
        print(new_lab_train_ds.shape)
        print(f' => DONE {len(new_lab_train_ds)}\n')

        print(f'Reducing the unlabeled train dataset {len(new_unlab_train_ds)}')
        for list_index, topk_index_value in overall_topk:
            new_unlab_train_ds = np.delete(new_unlab_train_ds, (list_index * n_samples) + topk_index_value, axis = 0)
            #new_unlab_train_ds = torch.cat((new_unlab_train_ds[ : ((list_index * n_samples) + topk_index_value)],
            #                                new_unlab_train_ds[((list_index * n_samples) + topk_index_value + 1) : ]))
        print(new_unlab_train_ds.shape)
        print(f' => DONE {len(new_unlab_train_ds)}\n')


        self.lab_train_ds = CIFAR10(None, new_lab_train_ds)
        self.unlab_train_ds = CIFAR10(None, new_unlab_train_ds)

        self.lab_train_dl = DataLoader(self.lab_train_ds, batch_size=batch_size, shuffle=True)
        self.unlab_train_dl = DataLoader(self.unlab_train_ds, batch_size=batch_size, shuffle=True)



    def train_AL_GTG(self, epochs, al_iters, gtg_tol, gtg_max_iter, top_k_obs, n_samples):
        iter = 0
        while iter < al_iters:
            print(f'----------------------- START ACTIVE LEARNING ITERATION {iter} -----------------------')
            self.__reintialize_model()
            self.fit(epochs) # train in the labeled observations
            self.get_unlabeled_samples(n_samples)
            self.get_embeddings('Labeled', self.lab_train_dl)

            ds_top_k = []

            for idx, (_, unlab_sample_dl) in enumerate(self.unlab_samp_list):

                print(f'----------- STARTING WORKING WITH UNLABELED SAMPLE # {idx} -----------')

                self.get_embeddings('Sampled', unlab_sample_dl)

                self.get_A()
                self.get_X()
                self.gtg(gtg_tol, gtg_max_iter)

                self.get_topK_obs(top_k_obs) # top k for the matrix X composed with the ds of labeled and unlabeled, so the index are referred to these two sets

                ds_top_k.append(self.topk_idx_val_obs)

                print(f'----------- ENDING WORKING WITH UNLABELED SAMPLE # {idx} -----------\n')

            overall_topk = get_overall_top_k(ds_top_k, top_k_obs)

            self.get_new_dataloader(overall_topk, n_samples)
            print(f'----------------------- END ACTIVE LEARNING ITERATION {iter} -----------------------\n')
            iter += 1


    def test_AL_GTG(self):
        val_loss, val_accuracy = self.evaluate(self.test_dl)

        print('TESTING RESULTS -> val_loss: {:.6f}, val_accuracy: {:.6f} \n'.format(val_loss, val_accuracy))



In [8]:
def accuracy_score(output, label):
    output_class = torch.argmax(torch.softmax(output, dim=1), dim=1)
    return (output_class == label).sum().item()/len(output)

In [9]:
def get_resnet18():
    resnet18 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', weights='DEFAULT')

    num_ftrs = resnet18.fc.in_features
    resnet18.fc = nn.Linear(num_ftrs, len(classes))

    return resnet18

In [10]:
splitted_train_dl, splitted_train_ds, train_dl, val_dl = get_initial_dataloaders(trainset, 0.2, 0.1)

resnet18 = get_resnet18()

optimizer = optim.SGD(resnet18.parameters(), lr=0.001, momentum=0.9)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=2, verbose=True)

epochs = 10

GTG_AL = GTG_ActiveLearning(
    n_classes = len(classes),
    model = resnet18,
    optimizer = optimizer,
    train_dl = train_dl,
    test_dl = test_dl,
    splitted_train_dl = splitted_train_dl,
    splitted_train_ds = splitted_train_ds,
    loss_fn = nn.CrossEntropyLoss(),
    val_dl = val_dl,
    score_fn = accuracy_score,
    scheduler = scheduler,
    device = device,
    patience = 5
)

GTG_AL.train_AL_GTG(epochs=epochs, al_iters=5, gtg_tol=0.001, gtg_max_iter=100, top_k_obs=80, n_samples=10) # top_k meglio che sia un multiplo di batch_size

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 82.3MB/s]


=> Saving Checkpoint to /content/init_checkpoint.pth.tar
 DONE

----------------------- START ACTIVE LEARNING ITERATION 0 -----------------------
=> Loading Initial Checkpoint
 DONE



  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/625 [00:00<?, ?it/s]

Epoch [1], train_loss: 1.782372, train_accuracy: 0.399000, val_loss: 1.380714, val_accuracy: 0.525600 

=> Saving Checkpoint to /content/best_checkpoint.pth.tar
 DONE



...

 => DONE

Obtaining Affinity Matrix
torch.Size([4320, 512]) torch.Size([3568, 512])
torch.Size([7888, 7888])
 => DONE with memory usage 248882176 bytes

Obtaining Initial X Matrix
torch.Size([7888, 10])
 => DONE with memory usage 315520 bytes

Runnning GTG Algorithm
 => DONE with 48 iterations

Obtaining the top_k most interesting observations
 => DONE

----------- ENDING WORKING WITH UNLABELED SAMPLE # 9 -----------

Copying the labeled train dataset


  0%|          | 0/4320 [00:00<?, ?it/s]

 => DONE

Copying the unlabeled train dataset


  0%|          | 0/35680 [00:00<?, ?it/s]

 => DONE

Expanding the labeled train dataset 4320
(4400, 2)
 => DONE 4400

Reducing the unlabeled train dataset 35680
(35600, 2)
 => DONE 35600

----------------------- END ACTIVE LEARNING ITERATION 4 -----------------------



In [11]:
GTG_AL.test_AL_GTG()

  0%|          | 0/625 [00:00<?, ?it/s]

TESTING RESULTS -> val_loss: 1.006619, val_accuracy: 0.667600 

